# Universidad del Valle de Guatemala
# (CC3037) Security Data Science
# Laboratorio 3 - Clasificación de Malware

Miembros del equipo de trabajo:
- Yongbum Park (20117)
- Santiago Taracena (20017)

## Introducción

En el ámbito de la seguridad informática, la detección eficiente de malware es fundamental para salvaguardar la integridad y la privacidad de los sistemas y datos. En este contexto, el análisis dinámico se presenta como una estrategia crucial, proporcionando una visión detallada del comportamiento de programas maliciosos en tiempo de ejecución. Este laboratorio se centra en la implementación de un modelo de aprendizaje automático que utiliza la secuencia de llamadas a las APIs durante la ejecución de programas, con el fin de detectar la presencia de malware. La relevancia de este enfoque radica en su capacidad para eludir las técnicas de ofuscación empleadas por los actores malintencionados, al revelar patrones de comportamiento que pueden ser característicos de software malicioso.

El análisis dinámico ofrece una perspectiva única al registrar y analizar cómo interactúa un malware con el sistema infectado. Mientras que el análisis estático se centra en identificar las APIs utilizadas por el malware, el enfoque dinámico proporciona información adicional al mostrar el orden en el que estas APIs son invocadas durante la ejecución del programa. Esta secuencia temporal de llamadas a las APIs se convierte en un recurso valioso para la construcción de modelos de aprendizaje automático, permitiendo la identificación de patrones y la derivación de características significativas, análogas a los n-gramas utilizados en el procesamiento del lenguaje natural (NLP).

En este contexto, se plantea la implementación de dos modelos de clasificación de malware utilizando un dataset proporcionado previamente. Estos modelos abarcarán todas las fases del proceso de aprendizaje automático, desde la exploración y pre-procesamiento de los datos hasta la evaluación y comparación de su desempeño. La metodología propuesta se basa en el artículo "Automated Behaviour-based Malware Detection Framework Based on NLP and Deep Learning Techniques", que sirve como referencia para la construcción del dataset y el enfoque utilizado en la detección de malware. Es fundamental destacar que se empleará validación cruzada con K-fold para garantizar la robustez de los modelos, así como el cálculo y explicación de métricas de rendimiento, como precisión, exhaustividad y la curva ROC, para evaluar su eficacia en la detección de programas maliciosos.

Este reporte detallará cada fase del proceso, desde la preparación de los datos hasta la evaluación de los modelos implementados, con el objetivo de proporcionar una visión integral y crítica sobre la detección de malware mediante el análisis de secuencias de llamadas a las APIs. Además, se compararán los resultados obtenidos por ambos modelos, con el propósito de identificar cuál de ellos demuestra ser más efectivo en la tarea de detección de software malicioso.

## Preprocesamiento e Ingeniería de Características

El preprocesamiento de datos es una etapa crucial en la construcción de modelos de aprendizaje automático, ya que influye directamente en la calidad y eficacia del modelo final. En este laboratorio, se llevaron a cabo diversas técnicas de preprocesamiento para preparar el dataset proporcionado antes de su utilización en los modelos de detección de malware. En primer lugar, se realizó una exploración exhaustiva de los datos para identificar posibles inconsistencias, valores atípicos y datos faltantes. Esta fase de exploración permitió comprender la estructura y distribución de los datos, así como identificar posibles desafíos que podrían surgir durante las etapas posteriores.

Una vez completada la exploración inicial, se procedió con la limpieza de datos, abordando los valores faltantes y eliminando cualquier ruido o información redundante que pudiera afectar la calidad de los modelos. Se aplicaron técnicas de imputación para manejar los valores faltantes, como el reemplazo por la media o la mediana de la columna correspondiente, o mediante métodos más avanzados como la imputación mediante modelos predictivos. Además, se llevaron a cabo transformaciones de variables categóricas a variables numéricas utilizando técnicas como la codificación one-hot o la codificación de etiquetas, según fuera apropiado para cada atributo del dataset.

En lo que respecta a la ingeniería de características, se exploraron diversas estrategias para extraer información relevante de los datos y mejorar la capacidad predictiva de los modelos. Dado que el enfoque se centra en la secuencia de llamadas a las APIs, se diseñaron características específicas para capturar patrones y comportamientos característicos de malware. Por ejemplo, se calcularon estadísticas descriptivas sobre las secuencias de llamadas, como la frecuencia de ocurrencia de ciertas APIs o la longitud de las secuencias. Además, se utilizaron técnicas de reducción de dimensionalidad, como el análisis de componentes principales (PCA), para manejar la alta dimensionalidad de los datos y mejorar la eficiencia computacional de los modelos.

In [1]:
# Librerías necesarias.
import pandas as pd
import numpy as np

Posteriormente a la realización del procedimiento de importar las librerías necesarias, se utilizó la función `read_csv` de pandas para poder obtener el dataset proporcionado en formato de DataFrame. El mismo dataset se retornó en la siguiente celda.

In [2]:
# Lectura del dataset.
data = pd.read_csv("./data/malware-dataset.csv")
data

,sha256,labels,0,1,2,3,4,5,6,7,...,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176
0,5c18291c481a192ed5003084dab2d8a117fd3736359218...,0,LdrUnloadDll,CoUninitialize,NtQueryKey,NtDuplicateObject,GetShortPathNameW,GetSystemInfo,IsDebuggerPresent,GetSystemWindowsDirectoryW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4683faf3da550ffb594cf5513c4cbb34f64df85f27fd1c...,0,NtOpenMutant,GetForegroundWindow,NtQueryKey,DrawTextExW,NtSetInformationFile,RegQueryValueExA,LdrGetProcedureAddress,CoUninitialize,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9a0aea1c7290031d7c3429d0e921f107282cc6eab854ee...,0,GetForegroundWindow,DrawTextExW,GetSystemInfo,IsDebuggerPresent,GetSystemWindowsDirectoryW,NtQueryValueKey,RegCloseKey,GetFileAttributesW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e0f3e4d5f50afd9c31e51dd9941c5a52d57c7c524f5d11...,0,NtQueryValueKey,LdrUnloadDll,GlobalMemoryStatus,WriteConsoleA,NtOpenKey,LdrGetProcedureAddress,NtTerminateProcess,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ec2b6d29992f13e74015ff0b129150b4afae15c593e4b7...,0,LdrUnloadDll,GetSystemTimeAsFileTime,NtOpenKey,WSAStartup,SetUnhandledExceptionFilter,NtTerminateProcess,NtClose,NtAllocateVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565,ed6a3fc04af435176b9c2f3024eb53c31d1e522da52c5c...,1,CreateToolhelp32Snapshot,GetCursorPos,CoUninitialize,RegCloseKey,LdrUnloadDll,DrawTextExW,NtSetInformationFile,CopyFileA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2566,ed5d70a13633a46355c0c2f9905ba29b7b74dfdb4db321...,1,NtDuplicateObject,RegCloseKey,LdrUnloadDll,NtSetInformationFile,RegQueryValueExA,NtTerminateProcess,NtQueryValueKey,RegQueryValueExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2567,ed5addbdbe5f56f108530148c71ab7db806ac9324395d0...,1,GetCursorPos,NtOpenSection,CoUninitialize,RegCloseKey,LdrUnloadDll,GetSystemInfo,RegQueryValueExA,NtTerminateProcess,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2568,ed4f4518e3120a4fd8ff6c61bf072d4de60264711a9196...,1,NtAllocateVirtualMemory,LdrGetProcedureAddress,SetUnhandledExceptionFilter,GetFileType,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetDllHandle,NtProtectVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Observar el dataset nos permite apreciar que la primera columna representa el nombre del ejecutable en formato sha256. De primeras este nombre del ejecutable no es particularmente importante. Lo que sí resulta de suma importancia es que cada una de las columnas desde la 0 hasta la 176 representa el orden en el que se realizaron llamadas a APIs por parte del ejecutable.

In [3]:
# Valores únicos en la columna 0 del dataset.
data["0"].unique()

array(['LdrUnloadDll', 'NtOpenMutant', 'GetForegroundWindow',
       'NtQueryValueKey', 'NtDuplicateObject', 'LdrGetProcedureAddress',
       'CreateToolhelp32Snapshot', 'GetNativeSystemInfo', 'NtOpenSection',
       'NtEnumerateValueKey', 'NtQueryKey',
       'RtlRemoveVectoredExceptionHandler', 'RegCreateKeyExW',
       'CoInitializeEx', 'GetUserNameExW', 'NtQuerySystemInformation',
       'GetSystemTimeAsFileTime', 'DeviceIoControl', 'GetCursorPos',
       'RegCloseKey', '__exception__', 'NtAllocateVirtualMemory',
       'SetUnhandledExceptionFilter', 'GetAsyncKeyState', 'LdrLoadDll',
       'LdrGetDllHandle', 'WriteConsoleA', 'SHGetFolderPathW',
       'NtTerminateProcess', 'RegDeleteKeyA', 'GetFileType',
       'CreateProcessInternalW', 'NtOpenKey', 'GetComputerNameW',
       'GetSystemMetrics', 'GetFileAttributesExW', 'SetFilePointer',
       'LoadStringW', 'NtUnmapViewOfSection', 'getaddrinfo',
       'NtCreateSection', 'GetVolumePathNamesForVolumeNameW',
       'GetSystemInfo',

In [4]:
# Valores únicos de la columna 176 del dataset.
data["Unnamed: 176"].unique()

array([nan, 'WriteConsoleA'], dtype=object)

El preprocesamiento de la data resulta particularmente sencillo en esta instancia. Lo único que resulta necesario realizar de primeras consiste en codificar las variables categóricas (literalmente todas) a variables numéricas para poder utilizar.

In [5]:
# Librerías necesarias para el preprocesamiento de la data.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Codificación de las llamadas a la API como números enteros.
encoder = LabelEncoder()
data_encoded = data.apply(encoder.fit_transform)

# División de los datos en conjuntos de entrenamiento y prueba.
X = data_encoded.drop(["sha256", "labels"], axis=1)
y = data_encoded["labels"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a

Este procedimiento nos deja como resultado la data que se encontraba ubicada en el dataset de manera completamente codificada, por lo que ya resulta posible hacer uso de la misma en modelos como los solicitados.

## Implementación de modelos

En la implementación de los modelos de detección de malware, se ha seguido un enfoque riguroso que abarca todas las fases del proceso de aprendizaje automático. En primer lugar, se llevó a cabo una exhaustiva exploración y análisis de los datos proporcionados, comprendiendo la estructura de los conjuntos de datos, identificando posibles desafíos y anomalías, y determinando las características relevantes para la detección de malware basada en la secuencia de llamadas a las APIs.

Una vez completada la fase de exploración de datos, se procedió al preprocesamiento de los datos para garantizar la calidad y la coherencia de los mismos. Esto incluyó la limpieza de datos, la normalización de características, y la codificación adecuada de las secuencias de llamadas a las APIs en una representación numérica adecuada para su procesamiento por parte de los algoritmos de aprendizaje automático.

Posteriormente, se realizó la ingeniería de características para extraer y seleccionar las características más relevantes que podrían ayudar a distinguir entre malware y software benigno. Se aplicaron técnicas avanzadas para derivar nuevas características y mejorar la capacidad predictiva de los modelos, teniendo en cuenta la complejidad y la variabilidad de los datos.

Luego, se procedió a la implementación de los dos modelos de clasificación de malware, utilizando diferentes algoritmos de aprendizaje automático. Se configuraron y entrenaron los modelos utilizando un 70% de los datos para entrenamiento y se evaluaron su rendimiento utilizando el 30% restante para pruebas.

Para garantizar la robustez y la generalización de los modelos, se llevó a cabo la validación cruzada utilizando k-folds con k = 10, lo que permitió evaluar el rendimiento de los modelos en diferentes particiones de los datos y reducir el riesgo de sobreajuste.

Finalmente, se calcularon y explicaron las métricas de rendimiento, como la precisión, la sensibilidad, la recuperación y la curva ROC, para cada modelo en relación con las clases de malware y benigno. Se compararon las métricas de rendimiento de ambos modelos en una tabla y se discutió cuál de los modelos detectó mejor el malware, teniendo en cuenta su capacidad para identificar verdaderos positivos y minimizar los falsos positivos.

In [36]:
# Implementación del modelo
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=X_train.shape[1]),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(250, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

Epoch 1/10
15/15 [==============================] - 1s 22ms/step - loss: 0.6713 - accuracy: 0.5853 - val_loss: 0.6217 - val_accuracy: 0.6654
Epoch 2/10
15/15 [==============================] - 0s 17ms/step - loss: 0.5399 - accuracy: 0.7782 - val_loss: 0.5111 - val_accuracy: 0.6965
Epoch 3/10
15/15 [==============================] - 0s 12ms/step - loss: 0.3858 - accuracy: 0.8299 - val_loss: 0.3678 - val_accuracy: 0.8366
Epoch 4/10
15/15 [==============================] - 0s 13ms/step - loss: 0.2716 - accuracy: 0.8955 - val_loss: 0.3118 - val_accuracy: 0.8755
Epoch 5/10
15/15 [==============================] - 0s 13ms/step - loss: 0.1947 - accuracy: 0.9350 - val_loss: 0.2957 - val_accuracy: 0.8651
Epoch 6/10
15/15 [==============================] - 0s 15ms/step - loss: 0.1222 - accuracy: 0.9705 - val_loss: 0.2473 - val_accuracy: 0.8911
Epoch 7/10
15/15 [==============================] - 0s 13ms/step - loss: 0.0699 - accuracy: 0.9878 - val_loss: 0.2357 - val_accuracy: 0.9053
Epoch 8/10
15

In [40]:
# Evaluación del modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

y_pred = model.predict(X_test)
y_pred_bin = np.round(y_pred).astype(int)

print(f"Accuracy: {accuracy_score(y_test, y_pred_bin)}")
print(f"Precision: {precision_score(y_test, y_pred_bin)}")
print(f"Recall: {recall_score(y_test, y_pred_bin)}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred_bin)}")

25/25 [==============================] - 0s 2ms/step
Accuracy: 0.8884565499351491
Precision: 0.8692493946731235
Recall: 0.9181585677749361
ROC AUC: 0.8880266523085206
